In [8]:
from frame_postprocess import *
import multiprocessing as mp

#### THIS SCRIPT COLLECTS THE RESPONSE FOR MULTIPLE SETS OF GROUND MOTIONS SCALED TO PARTICULAR INTENSITIES

In [9]:
##### Bldg and frame direction #####
bldg_name = 'Example_Bldg1'
####################################

# Data collection inputs
minrdrift = 5e-4 # Minimum residual drift to consider
fracElement = True # Boolean to collect or not fracture element results ###########CHANGE TO COLLECT FRACTURES###########
splice_frac_strain = 2*60/29000 # strain limit to judge that fracture occured in the splice
rdrift_out = 'all' # 'max' = only collect the peak RID
                   # 'all' = collects RID for every story
drift_out = 'both' # 'abs' = multiple columns with the peak absolute value for each floor
                   # 'both' = multiple columns with the peak positive an negative value for each floor

# Folder paths
results_folder = posixpath.join('1_Raw_NLRHA_results')
save_folder = posixpath.join('2_Collected_NLRHA_results')

# load cases inputs to locate data to collect
input_filepath = posixpath.join(results_folder, 'building_info_for_plot.csv')
inputs = pd.read_csv(input_filepath)

# Get variables
n_cases = len(inputs)

In [10]:
inputs

,OBJECTID,splice,beam_list,column_list,pz_list,colSplice,storyHeight,bayWidth,Tcond,HC_maf,...,meanM_225,meanM_475,meanM_975,meanM_2475,meanR_72,meanR_140,meanR_225,meanR_475,meanR_975,meanR_2475
0,1,1,"[[1,1,1,1,1,0,1,1,1],[1,1,1,1,1,1,1,1,1],[1,1,...","[[1,1,1,1,1,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,1],[...","[[0,0,0,0,0,0,0,0,0,0],[1,1,1,1,1,1,1,1,1,1],[...","[[0,0,0,0,0,0,0,0,0,0],[1,1,1,1,1,1,1,1,1,1],[...","[168,156,156,156,156,156,156,120,120,120,120,1...","[360,360,360,360,360,360,360,360,360]",2.21421,"[0.27213836,0.16862609,0.10778856,0.074009987,...",...,7.52,7.6275,7.6975,7.7675,38.0215,28.6287,24.0991,19.7812,17.4821,15.9414


# COLLECT EDP RESULTS

### STEP 1: Parse each stripe per case per stripe for paralell data collection

In [11]:
# Initialize data (for EDP table) -> these lists include cases x stripe
model_name_all          = []
stripe_folder_all       = []
save_results_folder_all = []
msa_folder_all          = []
beam_list_all           = []

splice_all              = []
splice_list_all         = []
column_list_all         = []
pz_list_all             = []

for case_i in range(n_cases):
    # Building model
    splice       = inputs.splice[case_i]

    # Building attributes
    beam_list   = np.array(eval(inputs.beam_list[case_i]))
    column_list = np.array(eval(inputs.column_list[case_i]))
    if splice:
        splice_list   = np.array(eval(inputs.colSplice[case_i]))
    
    # Save folderpath
    save_results_folder = posixpath.join(save_folder, bldg_name)
    try:
        os.mkdir(save_results_folder)
    except:
        print('Folder '+bldg_name+' already exists')
    
    # Folder names with results on specified directions
    
    # MSA results
    msa_folder = posixpath.join(results_folder, 'AnalysisResult', 'MSA')
    stripe_folders = os.listdir(msa_folder)
    n_stripes = len(stripe_folders)

    # Save data for each case to run in parallel (ONLY COLLECT IF RESULTS DON'T EXIST)
    for stripe in stripe_folders:
        
        results_filename = posixpath.join(save_results_folder, bldg_name + '_' + stripe + '.csv')
        
        if not os.path.isfile(results_filename):
            model_name_all.append(bldg_name)
            stripe_folder_all.append(stripe)
            save_results_folder_all.append(save_results_folder)
            msa_folder_all.append(msa_folder)
            beam_list_all.append(beam_list)
            splice_all.append(splice)            
            if splice:
                splice_list_all.append(splice_list)
                column_list_all.append(column_list)
            else:          
                splice_list_all.append(np.zeros(column_list.shape))
                column_list_all.append(np.zeros(column_list.shape))

FileNotFoundError: [WinError 3] The system cannot find the path specified: '1_Raw_NLRHA_results/AnalysisResult/MSA'

In [5]:
n_stripsBycases = len(model_name_all)
print('TOTAL PARALLEL JOBS TO COLLECT = ' + str(n_stripsBycases))
print('TOTAL CORES = ' + str(mp.cpu_count()))

TOTAL PARALLEL JOBS TO COLLECT = 6
TOTAL CORES = 32


### STEP 2: Collect RHA results in parallel (one job per stripe in all selected cases)

In [6]:
# folder = posixpath.join(msa_folder_all[2], stripe_folder_all[2], 'RSN266_VICT_CHI102')
# folder
# n_stories = 42

# pid_gm_p = get_EDPstory_response(folder, n_stories, 'drift_env_P')
# pid_gm_n = get_EDPstory_response(folder, n_stories, 'drift_env_n')
# pid_gm = get_EDPstory_response(folder, n_stories, 'drift_env')
# pd.DataFrame(np.hstack((pid_gm_p, pid_gm_n, pid_gm)))

# get_EDPstory_response(folder, n_stories, 'acc_env')

In [7]:
# collect_single_response(model_name_all, stripe_folder_all, save_results_folder_all, msa_folder_all, beam_list_x_all,
#                            beam_list_y_all, fracElement, splice_all, splice_list_x_all, splice_list_y_all, column_list_x_all, 
#                            column_list_y_all, minrdrift, splice_frac_strain, drift_out, rdrift_out, 0)

In [8]:
pool = mp.Pool(mp.cpu_count())
part_collector = partial(collect_single_response, model_name_all, stripe_folder_all, save_results_folder_all, msa_folder_all, beam_list_x_all,
                           beam_list_y_all, fracElement, splice_all, splice_list_x_all, splice_list_y_all, column_list_x_all, 
                           column_list_y_all, minrdrift, splice_frac_strain, drift_out, rdrift_out)
pool.map(part_collector, [i for i in range(n_stripsBycases)])
pool.close()
pool.join()
print('EDPs COLLECTED AND FILES CREATED')

[None, None, None, None, None, None]

EDPs COLLECTED AND FILES CREATED


### STEP 2: Check number of non-convergence runs

In [33]:
rp_list = ['72', '140', '225', '475', '975', '2475']
convergenceIsCol = True

In [34]:
results = {}

for case_i, model_name in enumerate(model_name_all):
    # Initilize dict to store stripe results for this case
    results_build = {}
    
    # Identify building name
    print(model_name)
    bldg_name = model_name.split('_')[0]
    
#     # Identify return periods
#     dirCase      = inputs.dir[case_i]
#     msa_folder = posixpath.join(results_folder, model_name, 'AnalysisResult', 'MSA', dirCase)
#     rp_list = os.listdir(msa_folder)
#     n_stripes = len(rp_list)
    
    # load results per stripe
    for rp in rp_list:        
        try:
            filename = 'EDP_' + model_name + '_' + rp +'.csv'
            results_path = posixpath.join(save_folder, bldg_name, filename)
            results_build[rp] = pd.read_csv(results_path)
        except:
            print('Skip rp=' + rp)
    results[model_name] = results_build

ID108_dirX_Constant_cvn12_a0_12_1splice


In [36]:
Pc   = {}
Pincon   = {}
n_collapses  = {}
n_gms  = {}
n_inconv = {}

for case_i, model_name in enumerate(model_name_all):   
    
    # Identify building name
    print(model_name)
    bldg_name = model_name.split('_')[0]
    
#     # Identify return periods
#     dirCase      = inputs.dir[case_i]
#     msa_folder = posixpath.join(results_folder, model_name, 'AnalysisResult', 'MSA', dirCase)
#     rp_list = os.listdir(msa_folder)
#     n_stripes = len(rp_list)
    
    # compute collapse fraction considering both direction
    pc  = np.zeros(len(rp_list))    
    n_collapses_rp  = np.zeros(len(rp_list))    
    n_gms_rp  = np.zeros(len(rp_list)) 
    n_inconv_rp = np.zeros(len(rp_list)) 
    pincon  = []
    
    for rp in list(results[model_name].keys()):
        rp_i = np.argwhere(np.array(rp_list) == rp)[0][0]
                       
        # Get RSN for each ground motion
        record_names = np.array(results[model_name][rp]['Unnamed: 0'])
#         record_names
        rsn = []
        for i in range(len(record_names)):
            rsn.append(record_names[i].split('_')[0])
        rsn = np.array(rsn)    
        
        # Get the end criteria for corresponding components
        endCriteria = np.array(results[model_name][rp]['EndCriteria'])
        
        # Compute collapse fractions
        if convergenceIsCol:
            # Considering non-convergence as collapse
            nonCollapse = endCriteria == 'nonCollapse'
            
            pc[rp_i] = (len(nonCollapse) - sum(nonCollapse))/len(nonCollapse)
            n_collapses_rp[rp_i] = len(nonCollapse) - sum(nonCollapse)
            n_gms_rp[rp_i] = len(nonCollapse)
            
            inconv_num = 0
            for i in range(len(endCriteria)):
                if endCriteria[i] == 'Inconvergence':
                    inconv_num += 1
            n_inconv_rp[rp_i] = inconv_num
            
            inconvergence = endCriteria == 'Inconvergence'
            pincon.append(sum(inconvergence)/len(inconvergence))                      
            
        else:
            col_num    = 0         
            total_num  = 0   
            inconv_num = 0
            for i in range(len(endCriteria)):
                if endCriteria[i] != 'Inconvergence':
                    total_num += 1
                    if endCriteria[i] == 'MaxDrift':
                        col_num += 1
                else:
                    inconv_num += 1
            
            pc[rp_i] = col_num/total_num
            n_collapses_rp[rp_i] = col_num
            n_gms_rp[rp_i] = total_num
            n_inconv_rp[rp_i] = inconv_num
    
    Pc[model_name]      = pc
    n_collapses[model_name] = n_collapses_rp
    n_gms[model_name]   = n_gms_rp
    n_inconv[model_name] = n_inconv_rp
    
    if convergenceIsCol:
        Pincon[model_name] = pincon
        
Pincon

ID108_dirX_Constant_cvn12_a0_12_1splice


{'ID108_dirX_Constant_cvn12_a0_12_1splice': [0.0,
  0.042857142857142858,
  0.14285714285714285,
  0.17142857142857143,
  0.34285714285714286,
  0.41428571428571431]}

# COLLECT ENDSTATE PER CASE

### STEP 1: Parse each stripe per case for paralell data collection

In [9]:
# Initialize data (inputs for endState results)
model_name_all          = []
stripe_folders_all      = []
save_results_folder_all = []
msa_folder_all          = []
beam_list_all         = []
column_list_all       = []
pz_list_all           = []
splice_all              = []
colSplice_all         = []

for case_i in range(n_cases):
    # Building model
    splice       = inputs.splice[case_i]

    # Building attributes
    beam_list   = np.array(eval(inputs.beam_list[case_i]))
    column_list = np.array(eval(inputs.column_list[case_i]))
    pz_list = np.array(eval(inputs.pz_list[case_i]))
    if splice:
        colSplice = np.array(eval(inputs.colSplice[case_i]))

    # Save folderpath (does not need to create it since previous block already did it)
    save_results_folder = posixpath.join(save_folder, bldg_name)
    
    # Folder names with results on both directions

    # MSA results
    msa_folder = posixpath.join(results_folder, 'AnalysisResult', 'MSA', dirCase)
    stripe_folders = os.listdir(msa_folder)
    n_stripes = len(stripe_folders)

    # Save data for each case to run in parallel
    model_name_all.append(model_name)
    stripe_folders_all.append(stripe_folders)
    save_results_folder_all.append(save_results_folder)
    msa_folder_all.append(msa_folder)
    beam_list_all.append(beam_list)
    column_list_all.append(column_list)
    pz_list_all.append(pz_list)
    splice_all.append(splice)            
    if splice:
        colSplice_x_all.append(colSplice)     
    else:            
        colSplice_x_all.append(np.zeros(column_list.shape))       

In [ ]:
def collect_endState_singleDir_response(model_name_all, save_results_folder_all, stripe_folders_all, msa_folders_all, beam_list_all,
                               column_list_all, pz_list_all, splice_all, colSplice_all, case_i):
# INPUTS
# All the inputs include information per case (different from the EDP collector that breaks each case into independent jobs per stripe
#    model_name_all           = list of str with the case name to collect results from
#    save_results_folder_all  = list of str with path to save results
#    stripe_folders_all       = list of list with the folder name of each stripe
#    msa_folders_all          = list eith the path_to_results
#    beam_list_all          = list of 2D np.array indicating which beams exist in the X frame
#    column_list_all        = list of 2D np.array indicating which columns exist in the X frame
#    pz_list_all            = list of 2D np.array indicating which pz exist in the X frame
#    splice_all               = list of boolean if splice are considered or ignored
#    colSplice_all          = list of 2D np.array indicating which stories have a splice in the X frame
#

# Parse case to execute
model_name    = model_name_all[case_i]
save_results_folder = save_results_folder_all[case_i]
stripe_folders= stripe_folders_all[case_i]
msa_folders   = msa_folders_all[case_i]
beam_list     = beam_list_all[case_i]
column_list   = column_list_all[case_i]
pz_list       = pz_list_all[case_i]
splice        = splice_all[case_i]
if splice == 1:
    colSplice = colSplice_all[case_i]

n_stripes = len(stripe_folders)
# Collect end state for every gm in every return period for BOTH DIRECTIONS

print('------- ' + model_name + ' -------')

# Select frame geometry matrices      
results_filename = posixpath.join(save_results_folder, model_name + '.h5')

# # count panel zones
n_stories, n_pier = column_list.shape
num_pz = 0
for i_story in range(n_stories):
    for i_pier in range(n_pier):
        if ((column_list[min(i_story + 1, n_stories-1), i_pier] == 1) or
                ((column_list[i_story, i_pier] == 1 )) and
                (beam_list[i_story, i_pier - 1])):
            existPZ = True
        elif ((column_list[min(i_story + 1, n_stories-1), i_pier] == 1) or
                ((column_list[i_story, i_pier] == 1 ) and
                 (beam_list[i_story, min(i_pier, n_pier -2)]))):
            existPZ = True
        else:
            existPZ = False

        if existPZ:
            num_pz += 1
print('num_pz='+str(num_pz))

# Removes existing file
if os.path.isfile(results_filename):
    os.remove(results_filename)
    print(results_filename + ' already exists, so deleted it')

# if True (Collects only data for those building without data)
if not os.path.isfile(results_filename):

    # Collect results and store in HDF file
    with h5py.File(results_filename, 'w') as hf:
        # prepare data groups per return period
        for group in stripe_folders:
            _ = hf.create_group('/' + group)

        # collect bldg response for each gm in each stripe
        for i in range(n_stripes):
            stripe_folder_path = posixpath.join(msa_folders, stripe_folders[i])

            print('RP = ' + str(stripe_folders[i]) + 'years')
            # print(stripe_folder_path)

            gm_ids = os.listdir(stripe_folder_path)
            for j in range(len(gm_ids)):
                # print(gm_ids[j])

                # collect results for this gm
                results_gm = posixpath.join(stripe_folder_path, gm_ids[j])

                # check if acc results available (gm finished?)
                pfa_gm = get_EDPstory_response(results_gm, n_stories, 'acc_env')
                if type(pfa_gm) == int:  # did not finish RHA, so skip the ground motion
                    print('Did not finish GM' + str(gm_ids[j]))
                else:
                    #    Panel zones
                    pz_response = get_pz_response(results_gm, pz_list, ['all_disp', 'pz_rot'])
                    #    beams and columns
                    column_response = get_column_response(results_gm, column_list, ['hinge_bot','hinge_top'])
                    beam_plas_rot = get_beam_response(results_gm, beam_list, ['hinge_left', 'hinge_right'])
                    frac_simulated  = get_beam_response(results_gm, beam_list, ['frac_LB','frac_LT','frac_RB','frac_RT'])
                    #    Splices
                    if splice == 1:
                        splice_response = get_splice_response(results_gm, colSplice, column_list, ['ss_splice'],
                                          res_type='Max', def_desired='strain')
                        splice_frac = splice_response['ss_splice'] > 2*60/29000

                    # create gm group
                    rp_group = hf['/' + stripe_folders[i]]
                    gm_record_group = rp_group.create_group(gm_ids[j])

                    # Save in h5 file's building_group
                    key = 'all_disp'
                    _ = gm_record_group.create_dataset(key, data=pz_response[key])
                    key = 'pz_rot'
                    _ = gm_record_group.create_dataset(key, data=pz_response[key])
                    key = 'hinge_bot'
                    _ = gm_record_group.create_dataset(key, data=column_response[key])
                    key = 'hinge_top'
                    _ = gm_record_group.create_dataset(key, data=column_response[key])
                    key = 'hinge_left'
                    _ = gm_record_group.create_dataset(key, data=beam_plas_rot[key])
                    key = 'hinge_right'
                    _ = gm_record_group.create_dataset(key, data=beam_plas_rot[key])
                    key = 'frac_LB'
                    _ = gm_record_group.create_dataset(key, data=frac_simulated[key])
                    key = 'frac_LT'
                    _ = gm_record_group.create_dataset(key, data=frac_simulated[key])
                    key = 'frac_RB'
                    _ = gm_record_group.create_dataset(key, data=frac_simulated[key])
                    key = 'frac_RT'
                    _ = gm_record_group.create_dataset(key, data=frac_simulated[key])
                    if splice == 1:
                        key = 'ss_splice'
                        _ = gm_record_group.create_dataset(key, data=splice_response[key])
                        key = 'splice_frac'
                        _ = gm_record_group.create_dataset(key, data=splice_frac)


### STEP 3: Collect end state of each building in parallel (one job per case)

In [10]:
# pool = mp.Pool(mp.cpu_count())
# part_collector = partial(collect_endState_singleDir_response, model_name_all, save_results_folder_all, stripe_folders_all, msa_folder_all, beam_list_all,
#                            column_list_all,  pz_list_all, splice_all, colSplice_all)
# pool.map(part_collector, [i for i in range(n_cases)])
# pool.close()
# pool.join()

case_i = 5
collect_endState_singleDir_response(model_name_all, save_results_folder_all, stripe_folders_all, msa_folders_all, beam_list_all,
                            column_list_all, pz_list_all, splice_all, 
                               colSplice_all, case_i)

------- ID108_dirX_Constant_cvn12_a0_12_1splice FRAME IN X -------
RP = 72years


Process ForkPoolWorker-47:
Process ForkPoolWorker-52:
Process ForkPoolWorker-61:
Process ForkPoolWorker-54:
Process ForkPoolWorker-48:
Process ForkPoolWorker-43:
Process ForkPoolWorker-34:
Process ForkPoolWorker-49:
Process ForkPoolWorker-37:
Process ForkPoolWorker-64:
Process ForkPoolWorker-39:
Process ForkPoolWorker-35:
Process ForkPoolWorker-42:
Process ForkPoolWorker-45:
Process ForkPoolWorker-40:
Process ForkPoolWorker-51:
Process ForkPoolWorker-50:
Process ForkPoolWorker-53:
Process ForkPoolWorker-44:
Process ForkPoolWorker-46:
Process ForkPoolWorker-36:
Process ForkPoolWorker-62:
Process ForkPoolWorker-41:
Process ForkPoolWorker-63:
Process ForkPoolWorker-38:
Process ForkPoolWorker-55:
Process ForkPoolWorker-57:
Process ForkPoolWorker-60:
Process ForkPoolWorker-56:
Process ForkPoolWorker-59:
Process ForkPoolWorker-58:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent 

  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._

  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/multiprocessing/qu

  File "/home/users/galvisf/aftershock_analysis/collect_nrha_results.py", line 2834, in collect_endState_single_response
    pz_response = get_pz_response(results_gm, pz_list, ['all_disp', 'pz_rot'])
  File "/home/users/galvisf/aftershock_analysis/collect_nrha_results.py", line 2214, in get_pz_response
    for line in f:
  File "/share/software/user/open/python/3.6.1/lib/python3.6/codecs.py", line 318, in decode
    def decode(self, input, final=False):
KeyboardInterrupt


KeyboardInterrupt: 

IndentationError: expected an indented block (<ipython-input-12-93572d8fa251>, line 17)